#### Интерпретация моделей машинного обучения

Модели машинного обучения (такие как нейронные сети, машины опорных векторов, ансамбли решающих деревьев) являются "прозрачными" в том смысле, что все происходящие внутри них вычисления известны. Но тем не менее часто говорят, что модели машинного обучения плохо интерпретируемы. Здесь имеется в виду то, что процесс принятия решения не удается представить в понятной человеку форме, то есть:

1. Понять, какие признаки или свойства входных данных влияют на ответ
2. Разложить алгоритм принятия решения на понятные составные части
3. Объяснить смысл промежуточных результатов, если они есть
3. Описать в текстовом виде алгоритм принятия решения (возможно, с привлечением схем или графиков)

Достичь полной интерпретируемости в машинном обучении, как правило, не удается, но даже частичная интерпретация может существенно помочь. Интерпретировав модель, мы можем открыть для себя что-то новое как о свойствах исследуемых данных, так и о свойствах обученной модели.

#### Интерпретация моделей для оценки качества

Авторы данной работы делают акцент на ситуации, когда требуется оценить качество обученной модели (например, чтобы сделать выбор между одной из нескольких моделей). Оценка точности на тестовой выборке не всегда позволяет хорошо оценить качество из-за следующих проблем:

1. Утечка данных (data leakage, или target leakage). Например, ID пациента может сильно коррелировать с диагнозом, но только в текущем датасете (который поделен на обучающую и тестовую часть). Модель, предсказывающая диагноз по ID, будет иметь высокую точность на тестовом датасете, но в целом очевидно, что в данной задаче такой способ предсказания некорректен и не будет хорошо работать на других данных. Избавиться от утечки данных не всегда просто.

> Еxample for this is KDD-Cup 2008 breast cancer prediction competition, where the patient ID contained an obvious leak. It is by no means obvious that removing this feature would leave a leakage-free dataset, however. Assuming different ID ranges corre spond to different health care facilities (in different geographical locations, with different equipment), there may be additional traces of this in the data. If for instance the imaging equipment's grey scale is slightly different and in particular grey levels are higher in the location with high cancer rate, the model without ID could pick up this leaking signal from the remaining data, and the performance estimate
would still be optimistic (the winners show evidence of this in their report) ([Kaufman et al., 2011]($Leakage in Data Mining: Formulation, Detection, and Avoidance$))

2. Сдвиг данных (data shift). Эта ситуация означает, что данные, на которых модель будет применяться, среднестатистически отличаются от тех, на которых модель обучалась и тестировалась. Это может привести к тому, что в обучающих и тестовых данных будут присутствовать такие корреляции, которые не будут обобщаться на другие выборки. Например, мы классифицируем животных по изображению, но большинство изображений рыб, которые у нас имеются, содержат также пальцы рыбака. Модель может научиться классифицировать как рыбу изображение, содержащее пальцы, что в целом неверно и может не работать на других выборках (см. также [Goyal and Bengio, 2020]($Inductive Biases for Deep Learning of Higher-Level Cognition$) и обзор [Априорные гипотезы и регуляризация в машинном обучении]($Априорные гипотезы и регуляризация в машинном обучении$); пример с рыбами см. в [Brendel and Bethge, 2019]($Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet$)).

Если мы узнаем, на что именно обращает внимание алгоритм, какими правилами руководствуется, то сможем оценить правдоподобность этих правил и тем самым оценить качество модели и распознать переобучение. Также мы сможем лучше понять, почему модель ошибается на тех или иных примерах, что может помочь улучшить алгоритм обучения.

> Humans usually have prior knowledge about the application domain, which they can use to accept (trust) or reject a prediction if they understand the reasoning behind it. ([Ribeiro et al., 2016]($“Why Should I Trust You?”: Explaining the Predictions of Any Classifier$))

Обзор способов интерпретации моделей машинного обучения можно найти, например, в [Linardatos et al., 2020]($Explainable AI: A Review of Machine Learning Interpretability Methods$) и [Li et al., 2021]($Interpretable Deep Learning: Interpretation, Interpretability, Trustworthiness, and Beyond$).

#### Интерпретация моделей на конкретном тестовом примере

Вместо попыток интерпретировать модель целиком, что может быть очень сложно, мы можем рассмотреть задачу интерпретации ответа модели $f$ на конкретном, фиксированном примере $x_0$. Например, если на данном изображении модель распознала собаку, то почему она распознала собаку? Какие части и свойства изображения повлияли на предсказание модели?

Для ответа на этот вопрос мы можем изменять $x_0$ и смотреть, как изменится при этом ответ модели, то есть мы изучаем зависимость $f(x_0 + \Delta x)$ от $\Delta x$. При этом возможно удастся с хорошей точностью аппроксимировать эту зависимость простой функцией $g(\Delta x)$. Такой подход называется локальной аппроксимацией модели вокруг точки $x_0$.

Например, рассчитав градиент $\nabla f(x)$ в точке $x_0$ мы узнаем, как изменится ответ при очень малых изменениях $\Delta x$. При этом мы получаем локальную линейную аппроксимацию (в курсе высшей математики такая аппроксимация называется дифференциалом функции $f$). Такой подход используется, например, при расчете так называемых saliency maps в компьютерном зрении ([Simonyan et al., 2013]($Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps$)) - производных выходных значений сети по отдельным пикселям изображения. Но такая аппроксимация далеко не всегда адекватна:

1. Производная локальна и не говорит о том, как изменится ответ при существенных изменениях $\Delta x$. Например, если один из признаков достиг состояния "насыщения", то есть значение данного признака более чем достаточно, чтобы сделать какой-то вывод о целевой переменной, то производная по нему почти равна нулю. Эффект будет лишь если мы сильно изменим данный признак.
2. В некоторых моделях (решающих деревьях) производная не имеет смысла: она либо равна нулю, либо не существует.
3. Для бинарных признаков производная также не имеет смысла, поскольку малое изменение признака ведет в "невозможную" область нецелого значения, в котором модель и не обязана работать корректно.

#### LIME (Local Interpretable Model-agnostic Explanations)

Авторы данной работы предлагают фиксировать тестовый пример $x_0$ и интерпретировать ответ модели $f(x_0)$ с помощью анализа значений $f(x_0 + \Delta x)$ для некоторого конечного набора значений $\Delta x$. Алгоритм LIME выглядит следующим образом:

**Шаг 1.** Выбираем модель $f: X \to Y$ и тестовый пример $x_0 \in X$.

**Шаг 2.** Нам нужно определить, как мы будем выбирать $\Delta x$. Идея в том, что это должны быть какие-то осмысленные изменения $x_0$. В алгоритме LIME мы должны выбрать $M$ и задать функцию $h_{x_0}: \{0, 1\}^M \to X$, при этом $h_{x_0}([1, 1, \dots, 1]) = x_0$. Вектор из множества $\{0, 1\}^M$ (обозначаемый $x^\prime$) при этом рассматривается как "упрощенное представление" объекта из $X$, что будет ясно из следующих примеров:

*Пример 1.* Пусть модель $f$ работает с изображениями. Мы можем поделить изображение $x_0$ на $M$ суперпикселей (областей, разделенных видимыми границами). $i$-й элемент вектора $x^\prime$ описывает наличие $i$-го суперпикселя. В исходном изображении $x_0$ присутствуют всех суперпиксели, поэтому вектор из единиц отображается в $x_0$. Произвольный вектор $x^\prime$ мы отображаем в $X$ функцией $h_{x_0}$ следующим образом: Все "пристствующие" суперпиксели рисуются без изменений. Все "отсутствующие" суперпиксели заполняются белым цветом (либо, как вариант, усредненным цветом соседних суперпикселей).

*Пример 2.* Пусть модель $f$ осуществляет классификацию текста методом bag-of-words, то есть в качестве входных данных принимает количество повторений тех или иных слов в тексте. Словарь модели состоит из $K$ слов, и $i$-й элемент входного вектора означает количество повторений в тексте $i$-го слова из словаря (таким образом, весь входной вектор $x_0 \in X$ имеет размерность $K$). Возьмем $M=K$ и выберем функцию $h_{x_0}$ таким образом: если $i$-й элемент $x^\prime$ равен 1, то  $i$-й элемент $h_{x_0}(x^\prime)$ (т. е. значение счетчика для $i$-го слова) равен $i$-му элементу $x_0$, в противном случае $i$-й элемент $h_{x_0}(x^\prime)$ равен нулю. Таким образом, вектор из единиц отображается в $x_0$. Зануляя некоторые элементы $x^\prime$, мы обнуляем счетчики для соответствующих слов из словаря.

Из примеров видно, что функция $h_{x_0}$ специфична для выбранного тестового примера $x_0$. Эту функцию мы можем выбрать произвольно, но так, чтобы отдельные элементы упрощенного представления $x^\prime$ были интерпретируемы. При этом мы надеемся, что объяснить предсказание модели на $x_0$ можно интерпретировать, изучая, как влияют на ответ изменения отдельных элементов вектора $x^\prime$. Конечно, функция $h_{x_0}$ может быть выбрана неудачно. Например, если модель определяет стиль фотографии как "ретро" при наличии оттенка "сепия", то изменения отдельных суперпикселей не помогут интерпретировать модель. Если функция $h_{x_0}$ была выбрана неудачно, то всегда можно попробовать заново с другой $h_{x_0}$.

Задав $h_{x_0}$, мы можем вычислить ответ модели для произвольного упрощенного представления $x^\prime$: $f(h_{x_0}(x^\prime))$. Если $M$ велико, то мы не можем перебрать все возможные $x^\prime$, поскольку их количество равно $2^M$. Вместо этого, нам придется ограничиться перебором небольшого числа значений: $x^\prime_{(1)}, x^\prime_{(2)}, \dots, x^\prime_{(N)}$ (нижний индекс здесь означает номер примера в выборке, а не номер элемента в векторе).

**Шаг 3.** Зададим функцию $\pi_{x_0}: \{0, 1\}^M \to \mathbb{R}$. Эта функция сопоставляет каждому $x^\prime$ число, описывающее меру близости $f(h_{x_0}(x^\prime))$ к $x_0$. Эти числа будут использоваться как веса при семплировании выборки $x^\prime_{(i)}$. Как вариант, авторы предлагают выбирать такую функцию:

$\pi_{x_0}(x^\prime) = \exp(-D(x_0, h_{x_0}(x^\prime))\ /\ \sigma^2)$,

где $D$ - метрика расстояния, например cosine similarity для bag-of-words и среднеквадратичное отклонение для изображений. Такая функция называется гауссовым радиально-базисным ядром и используется также в [SVM]($SVM и ядерные методы в машинном обучении$). Легко видеть ее сходство с формулой плотности вероятности нормального распределения.

Идея в том, что в первую очередь мы хотели бы перебрать значения $x^\prime$, близкие к вектору из единиц: это соответствует небольшим изменениям в $x_0$ (чем больше нулей в $x^\prime$, тем более сильное изменение $x_0$ мы рассматриваем). Поэтому функция $\pi_{x_0}$ должна сопоставлять большие веса векторам, в которых много единиц и мало нулей.

*Примечание.* В одной из последующих работ ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$)) было предложено использовать в качестве $\pi_{x_0}$ функцию, назначающую большие веса как векторам с большим количеством единиц, так и векторам с большим количеством нулей. Такую функцию авторы назвали Shapley kernel (см. раздел 4.1 и иллюстрацию 2A). Семплируя значения $x^\prime_{(i)}$ с помощью Shapley kernel, мы акцентируем внимание в том числе на значениях $x^\prime$ с большим количеством нулей, то есть на отдельных интерпретируемых компонентах (напрмер, отдельных суперпикселях в случае изображений).

**Шаг 4.** Соберем датасет из измененных (относительно $x_0$) примеров. Для этого выберем вектора $\{ x^\prime_{(i)} \}_{i=1}^N$ из равномерного распределения и отображаем эти примеры в $X$: $x_{(i)} = h_{x_0}(x^\prime_{(i)})$. Также присвоим примерам веса $w_i = \pi_{x_0}(h_{x_0}(x^\prime_{(i)}))$. Эти примеры являются тем набором значений $x + \Delta x$, на которых мы хотим проверить работу модели (см. первый абзац данного раздела).

**Шаг 5.** Определим модель $g: \{0, 1\}^M \to Y$, которую будем использовать для локальной аппроксимации. Эта модель должна быть интерпретируемой, например, линейной моделью или решающим деревом. Авторы рассматривают в основном случай линейной модели. В следующей формуле нижний индекс обозначает элемент вектора $x^\prime$:

$g(x^\prime) = \phi_0 + \sum\limits_{i=0}^M \phi_i x^\prime_i$

Таким образом, мы пытаемся аппроксимировать предсказание модели $f$ в окрестности $x_0$ моделью $g$. Значение $\phi_i$ означает вес, присваиваемый $i$-му элементу упрощенного представления (например, $i$-му суперпикселю в случае изображений). При этом мы предполагаем, что такая аппроксимация адекватна, то есть наличие или отсутствие $i$-го элемента упрощенного представления линейно влияет на ответ модели. Если окажется, что это не так (например, если модель $g$ не сможет хорошо обучиться), то мы можем попробовать другую функцию $h_{x_0}$ или другую модель $g$.

Для обучения модели $g$ осталось выбрать функцию потерь $\mathcal{L}$ - например, среднеквадратичное отклонение. Эта функция будет сравнивать предсказания моделей $f$ и $g$ на собранном датасете. Авторы предлагают также использовать "штраф за сложность" $\Omega(g)$ - например, количество ненулевых весов в линейной модели.

**Шаг 6.** Обучим модель $g$, минимизируя функцию потерь. Например, если в качестве функции потерь выбрано среднеквадратичное отклонение, то задача оптимизации формулируется следующим образом (см. также формулу 2 в разделе 3.4 статьи):

$\sum\limits_{i=1}^N w_i (f(x_{(i)}) - g(x^\prime_{(i)}))^2 + \Omega(g) \to \min\limits_g$

В данной формуле первым слагаемым является сумма среднеквадратичных отклонений по всему датасету, при этом примерам сопоставляются веса $w_i$, рассчитанные с помощью $\pi_{x_0}$ (см. шаг 4).

**Резюме.** Суть подхода LIME в том, что мы аппроксимируем предсказание модели $f$ в окрестности тестового примера $x_0$ более простой, легко интерпретируемой моделью $g$ (например, линейной моделью). При этом модель $g$ использует в качестве входных данных упрощенное представление тех данных, которые использует модель $f$ (см. шаг 2). Выбор способа формирования упрощенного представления является ключевым шагом в LIME (см. примеры выше).

Если модель $g$ плохо обучается (функция потерь остается высокой), то вероятно мы выбрали не лучший способ формирования упрощенного представления. Например, в случае изображений это может означать, что мы не можем линейно влиять на предсказание модели, удаляя отдельные суперпиксели. Возможно, модель ориентируется не на отдельные объекты, а на цвет изображения в целом. Тогда можно попробовать использовать другое упрощенное представление, элементами которого является информация об усредненном цвете изображения.